In [7]:
import matplotlib

matplotlib.use('TkAgg')
import LDP.basicDP.OLHbasic as lhb
import LDP.basicFunction.basicfunc as bf
import LDP.basicDP.LDPKVbasic as lkvb
import numpy as np
import random
import time

"""数据处理"""
# 关闭科学计数法显示
np.set_printoptions(suppress=True)

#### 读取文件
#### 构建kv对元组列表，每一行代表一个用户的kv对list

In [8]:
k_path='../E_Commerce/data/Ecomm_k.txt'
v_path = '../E_Commerce/data/Ecomm_v.txt'
lable_path = '../E_Commerce/data/Ecomm_lable.txt'

data_k = bf.readtxt(k_path)
data_v = bf.readtxt(v_path)
data_l = bf.readtxt(lable_path)

n = len(data_k)  # 用户数量
l = len(data_l[0])  # key域的长度
epsilon = 5

# 构建kv对元组列表，每一行代表一个用户的kv对list
kv = []

for i in range(n):
    data_k[i]=list(map(int,data_k[i]))
    tmp = zip(data_k[i], data_v[i])
    kv.append(list(tmp))
  
# 构建label标签，我已经把标签规范化为[1,l]的数字了，也就是说不需要用之前的原始标签了
label=[i+1 for i in range(l)]

## 1.采样
#### 注意！！！！要转换为和samplek相同的格式，因为hash是专为str计算的，int和float转换得到的结果不同

一维数据不需要采样！，下面注释掉了

In [9]:
# samplekv = []
# for i in range(n):
#     random.seed(1)
#     tmp = random.sample(kv[i], 1)
#     samplekv.append(tmp[0])
# samplekv_unzip = list(zip(*samplekv))
# samplek = samplekv_unzip[0]

samplek=[d[0] for d in data_k]
samplek

[768,
 1081,
 1078,
 1050,
 848,
 1081,
 859,
 859,
 1078,
 1078,
 1078,
 1096,
 1096,
 768,
 1078,
 1066,
 1066,
 854,
 1121,
 1078,
 848,
 1081,
 1078,
 1078,
 848,
 698,
 950,
 1004,
 685,
 5,
 1061,
 1061,
 1061,
 950,
 698,
 1061,
 1003,
 950,
 685,
 863,
 863,
 863,
 911,
 90,
 863,
 863,
 863,
 824,
 911,
 863,
 863,
 863,
 1105,
 863,
 863,
 863,
 369,
 863,
 1096,
 369,
 1096,
 369,
 863,
 863,
 863,
 863,
 863,
 863,
 863,
 1079,
 846,
 823,
 851,
 867,
 1082,
 846,
 1083,
 851,
 837,
 1082,
 1073,
 864,
 1134,
 823,
 862,
 867,
 851,
 1082,
 846,
 837,
 1079,
 851,
 862,
 1082,
 851,
 864,
 846,
 862,
 1134,
 862,
 862,
 864,
 823,
 823,
 864,
 837,
 837,
 837,
 837,
 1082,
 862,
 1082,
 837,
 1082,
 1083,
 864,
 1073,
 1079,
 851,
 837,
 862,
 967,
 1197,
 837,
 924,
 1073,
 1134,
 1134,
 967,
 864,
 846,
 1082,
 862,
 967,
 846,
 862,
 967,
 837,
 127,
 1009,
 830,
 829,
 1127,
 1009,
 830,
 829,
 1009,
 1012,
 1009,
 1009,
 1009,
 1009,
 830,
 830,
 830,
 830,
 1009,
 524

In [10]:
# 保存采样结果，之后跑实验都直接用这个采样samplek直接开始进行，因为采样我加了种子
# 采样得到的结果都一样
# 这里是因为本来就是一维数据，根本不需要采样，也不会变的

# flag为False 将数据保存成一行
bf.savetxt(samplek,"../E_Commerce/data/Ecomm_samplek.txt",flag=False)

保存文件成功


#### samplek的真实计数结果

In [5]:
# count_true = []  # 真实计数结果
# for i in range(l):
#     count_true.append((label[i], samplek.count(label[i])))
# 
# count_true

[(1, 1),
 (2, 3),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 7),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 8),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 3),
 (37, 1),
 (38, 4),
 (39, 3),
 (40, 2),
 (41, 3),
 (42, 1),
 (43, 3),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 3),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 2),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 2),
 (61, 1),
 (62, 4),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 2),
 (68, 1),
 (69, 2),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 2),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 14),
 (92, 1),
 (93, 1),
 (94, 2),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),
 (101, 

In [6]:
# samplekpath='../E_Commerce/result/samplekcount.txt'
# bf.savetxt(count_true,samplekpath)

保存文件成功


## 2.用OLH查找候选集

In [11]:
starttime = time.clock()
print("开始执行OLH...")

# 调用OLH
est=lhb.OLH_1(epsilon,samplek,n,l)

# 程序运行结束时间
endtime = time.clock()
print('运行时间:', endtime - starttime)

est

开始执行OLH...
运行时间: 20.5119848


[(1, 8.858),
 (2, -3.288),
 (3, -5.312),
 (4, 10.883),
 (5, 0.761),
 (6, -1.264),
 (7, -3.288),
 (8, -19.483),
 (9, -31.629),
 (10, -33.653),
 (11, -25.556),
 (12, 43.272),
 (13, 0.761),
 (14, 6.834),
 (15, 67.564),
 (16, -9.361),
 (17, -43.775),
 (18, 16.956),
 (19, 16.956),
 (20, -1.264),
 (21, -1.264),
 (22, -15.434),
 (23, 14.931),
 (24, -25.556),
 (25, 4.81),
 (26, 8.858),
 (27, 12.907),
 (28, -7.337),
 (29, -33.653),
 (30, 0.761),
 (31, -29.604),
 (32, 39.223),
 (33, 4.81),
 (34, -11.385),
 (35, -23.531),
 (36, -3.288),
 (37, 14.931),
 (38, -49.848),
 (39, 6.834),
 (40, -25.556),
 (41, 8.858),
 (42, 16.956),
 (43, -57.945),
 (44, -7.337),
 (45, -13.41),
 (46, -23.531),
 (47, -15.434),
 (48, 16.956),
 (49, 6.834),
 (50, 4.81),
 (51, 55.418),
 (52, -11.385),
 (53, 65.54),
 (54, 33.15),
 (55, 8.858),
 (56, 6.834),
 (57, 23.029),
 (58, 12.907),
 (59, 12.907),
 (60, -19.483),
 (61, -35.677),
 (62, -5.312),
 (63, 4.81),
 (64, 21.004),
 (65, -19.483),
 (66, -11.385),
 (67, -7.337),
 (68

In [12]:
# 保存采样结果
bf.savetxt(est,'../E_Commerce/result/sample_est.txt')

保存文件成功


#### 生成top_k的候选集(候选集长度为2k)

In [13]:
k=20
est.sort(key=lambda x:x[1],reverse=True)
topk_candi=est[:2*k]


#在不会出现相同key的情况下，可以转为dict，提升速度
candi_dict=dict(topk_candi)
candi_k=list(candi_dict.keys())
candi_k

[1079,
 863,
 1095,
 1082,
 873,
 830,
 1111,
 869,
 896,
 851,
 1096,
 1078,
 861,
 868,
 937,
 862,
 864,
 1081,
 1060,
 1087,
 874,
 1034,
 1057,
 1023,
 1073,
 1084,
 855,
 928,
 832,
 837,
 1105,
 829,
 1093,
 821,
 834,
 836,
 1026,
 835,
 1009,
 876]

In [14]:
# 保存候选集，长度为2k
bf.savetxt(candi_k,'../E_Commerce/result/candidate.txt')


保存文件成功


#### 与每个用户的项集kv取交集得到用户本地项集minikv

In [15]:
minik=[]
miniv=[]
minikv=[]
# 每个用户的交集项个数
numsize=[]
for i in range(n):
    # data_k[i]是每个用户的项集的k值
    # 将data_k[i]与候选集candi_k取交集得到tmpk
    tmpk=list(set(candi_k).intersection(set(data_k[i])))
    # 根据tmpk得到对应项的v值tmpv
    di=dict(kv[i])
    tmpv=list(map(di.get,tmpk))
    minik.append(tmpk)
    miniv.append(tmpv)
    minikv.append(list(zip(tmpk,tmpv)))
    numsize.append(len(tmpk))
    
numsize

[0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [17]:
# 保存用户取交集之后的本地项集minik和miniv
bf.savetxt(minik,'../E_Commerce/result/minik.txt')
bf.savetxt(miniv,'../E_Commerce/result/miniv.txt')

保存文件成功
保存文件成功


## 3.用OLH上传交集的项数

# 一维数据这里可以省略，因为L肯定只能为1，可以省掉一部分隐私预算

In [12]:
# starttime = time.clock()
# print("开始执行OLH...")
# numsize_lable=[i for i in range(2*k+1)]
# # 调用OLH
# numsize_est = lhb.OLH(epsilon, numsize, n, numsize_lable)
# 
# # 程序运行结束时间
# endtime = time.clock()
# print('运行时间:', endtime - starttime)
# 
# numsize_est

开始执行OLH...
运行时间: 4.137952000000041


[(0, 9408.204),
 (1, 7189.516),
 (2, 7853.503),
 (3, 7942.574),
 (4, 7891.966),
 (5, 6978.983),
 (6, 5863.566),
 (7, 5727.935),
 (8, 5412.136),
 (9, 5442.501),
 (10, 5122.654),
 (11, 5316.991),
 (12, 5252.212),
 (13, 5049.777),
 (14, 4774.465),
 (15, 4533.568),
 (16, 3889.824),
 (17, 3725.852),
 (18, 3401.956),
 (19, 3181.301),
 (20, 2721.774),
 (21, 2495.046),
 (22, 2195.442),
 (23, 1984.91),
 (24, 1845.23),
 (25, 1626.6),
 (26, 1444.408),
 (27, 1361.41),
 (28, 1282.46),
 (29, 1302.704),
 (30, 1130.634),
 (31, 1033.465),
 (32, 920.101),
 (33, 820.908),
 (34, 594.181),
 (35, 598.23),
 (36, 450.452),
 (37, 436.282),
 (38, 298.626),
 (39, 280.407),
 (40, 278.382)]

In [ ]:
#### 获得L值

In [13]:
# d=dict(numsize_est)
# s=sum(list(d.values()))
# target=0.9*s
# target

125155.02240000003

In [19]:
#一维数据L只能为1，不用算
L=1

## 4.Padding and Sampling

In [20]:
padlabel=[i+1 for i in range(l,l+L)]
cp_label=candi_k+padlabel
upkv=[lkvb.P_S(minikv[i],l,L) for i in range(n)]

upkv

[(1207, -1),
 (1081, 1),
 (1078, 1),
 (1207, -1),
 (1207, -1),
 (1081, -1),
 (1207, -1),
 (1207, -1),
 (1078, 1),
 (1078, 1),
 (1078, 1),
 (1096, 1),
 (1096, 1),
 (1207, 1),
 (1078, 1),
 (1207, -1),
 (1207, -1),
 (1207, -1),
 (1207, -1),
 (1078, 1),
 (1207, 1),
 (1081, 1),
 (1078, -1),
 (1078, 1),
 (1207, -1),
 (1207, 1),
 (1207, 1),
 (1207, 1),
 (1207, -1),
 (1207, 1),
 (1207, 1),
 (1207, -1),
 (1207, -1),
 (1207, -1),
 (1207, 1),
 (1207, 1),
 (1207, 1),
 (1207, -1),
 (1207, 1),
 (863, 1),
 (863, 1),
 (863, 1),
 (1207, -1),
 (1207, -1),
 (863, 1),
 (863, 1),
 (863, 1),
 (1207, -1),
 (1207, -1),
 (863, 1),
 (863, 1),
 (863, 1),
 (1105, 1),
 (863, 1),
 (863, 1),
 (863, 1),
 (1207, 1),
 (863, 1),
 (1096, 1),
 (1207, 1),
 (1096, 1),
 (1207, 1),
 (863, 1),
 (863, 1),
 (863, 1),
 (863, 1),
 (863, -1),
 (863, 1),
 (863, 1),
 (1079, 1),
 (1207, -1),
 (1207, -1),
 (851, 1),
 (1207, -1),
 (1082, 1),
 (1207, 1),
 (1207, -1),
 (851, -1),
 (837, 1),
 (1082, -1),
 (1073, 1),
 (864, 1),
 (1207, 1),


In [21]:
bf.savetxt(upkv,'../E_Commerce/result/upkv.txt')
bf.savetxt(cp_label,'../E_Commerce/result/cp_label.txt')

保存文件成功
保存文件成功


#### UE扰动


In [22]:
d=2*k
eps=10
eps1=np.log((np.exp(eps)+1)/2)
eps2=eps
a=0.5
p=lkvb.p_values(eps2)
b=1-lkvb.p_values(eps1)

In [23]:
Y=lkvb.PCKV_UE(upkv,d,L,cp_label,a,p,b)

####聚合

In [24]:
f_k,m_k=lkvb.AEC_UE(Y,d,L,n,a,p,b)
print(f_k)
print(m_k)

[0.04172330143886572, 0.03201362730892445, 0.031843282148750034, 0.023837059620552846, 0.023070506399768008, 0.020430156417064676, 0.02051532899715188, 0.017363943533925327, 0.019322912875931023, 0.012849796789303505, 0.014042212910524364, 0.012764624209216301, 0.011657380668082647, 0.013871867750349955, 0.01336083226982673, 0.010039101646425767, 0.013020141949477914, 0.012594279049041892, 0.011998070988431463, 0.011146345187559421, 0.011146345187559421, 0.01029461938668738, 0.009528066165902542, 0.0086763403650305, 0.009442893585815338, 0.010890827447297809, 0.0066321984429375995, 0.008505995204856091, 0.006376680702675987, 0.008931858105292113, 0.007824614564158458, 0.010209446806600176, 0.009783583906164154, 0.009613238745989745, 0.006717371023024804, 0.005184264581455129, 0.005524954901803945, 0.007483924243809641, 0.008420822624768887, 0.005099092001367924]
[0.6858371166027147, 0.654426075446637, 0.6151349232587316, 0.6716848294839716, 0.7641411559551645, 0.6586364601168009, 0.643

In [26]:
lable_path = '../E_Commerce/data/Ecomm_id_count_avg.txt'

id_count= bf.readtxt(lable_path)
id=[]
true_count=[]
true_avg=[]
for i in candi_k:
    tmp=id_count[int(i)-1]
    id.append(tmp[0])
    true_count.append(tmp[1])
    true_avg.append(tmp[2])
true_count

[1024.0,
 806.0,
 756.0,
 582.0,
 545.0,
 527.0,
 480.0,
 430.0,
 404.0,
 338.0,
 327.0,
 297.0,
 288.0,
 351.0,
 358.0,
 244.0,
 306.0,
 289.0,
 294.0,
 291.0,
 238.0,
 220.0,
 213.0,
 205.0,
 197.0,
 249.0,
 152.0,
 214.0,
 164.0,
 205.0,
 185.0,
 225.0,
 220.0,
 211.0,
 157.0,
 141.0,
 125.0,
 150.0,
 186.0,
 115.0]

In [27]:
bf.savetxt(id,'../E_Commerce/result/id.txt')
bf.savetxt(true_count,'../E_Commerce/result/true_count.txt')
bf.savetxt(true_avg,'../E_Commerce/result/true_avg.txt')

保存文件成功
保存文件成功
保存文件成功


In [28]:
est_count=[]
for i in f_k:
    est_count.append(i*n)
bf.savetxt(est_count,'../E_Commerce/result/est_count.txt')


保存文件成功
